In [ ]:
!pip install nltk
!pip install rake_nltk
!pip install wget
!pip install keybert

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import pandas as pd

json_data = pd.read_json('/content/drive/MyDrive/HackOff Project/Cell_Phones_and_Accessories_5.json', lines = True)
json_data

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"
...,...,...,...,...,...,...,...,...,...
194434,A1YMNTFLNDYQ1F,B00LORXVUE,eyeused2loveher,"[0, 0]",Works great just like my original one. I reall...,5,This works just perfect!,1405900800,"07 21, 2014"
194435,A15TX8B2L8B20S,B00LORXVUE,Jon Davidson,"[0, 0]",Great product. Great packaging. High quality a...,5,Great replacement cable. Apple certified,1405900800,"07 21, 2014"
194436,A3JI7QRZO1QG8X,B00LORXVUE,Joyce M. Davidson,"[0, 0]","This is a great cable, just as good as the mor...",5,Real quality,1405900800,"07 21, 2014"
194437,A1NHB2VC68YQNM,B00LORXVUE,Nurse Farrugia,"[0, 0]",I really like it becasue it works well with my...,5,I really like it becasue it works well with my...,1405814400,"07 20, 2014"


Generate stopwords

In [ ]:
import nltk
from pathlib import Path
import wget

my_stopwords = set()
my_stopwords.update(nltk.corpus.stopwords.words('english'))
my_stopwords.update(["one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten",
                     "eleven", "twelve", "thirteen", "fourteen", "fifteen", "sixteen", "seventeen", "eighteen", "nineteen", "twenty"
                     "hundred"])

# ignore positive adjective words
if not Path('/content/positive-words.txt').exists():
  wget.download('https://gist.githubusercontent.com/mkulakowski2/4289437/raw/1bb4d7f9ee82150f339f09b5b1a0e6823d633958/positive-words.txt', '/content/positive-words.txt')
with open('/content/positive-words.txt', 'r') as positive_words:
  for line in positive_words:
    stripped_str = line.strip()
    if not stripped_str.startswith(';') and len(stripped_str) > 0:
      my_stopwords.add(stripped_str)

# ignore negative adjective words
if not Path('/content/negative-words.txt').exists():
  wget.download('https://gist.githubusercontent.com/mkulakowski2/4289441/raw/dad8b64b307cd6df8068a379079becbb3f91101a/negative-words.txt', '/content/negative-words.txt')
with open('/content/negative-words.txt', 'r') as negative_words:
  for line in negative_words:
    stripped_str = line.strip()
    if not stripped_str.startswith(';') and len(stripped_str) > 0:
      my_stopwords.add(stripped_str)
    
with open('stopfile.txt', 'w', newline='\n') as stopfile:
    stopfile.truncate(0)
    for word in my_stopwords:
      stopfile.write(word + "\n")
  

In [ ]:
import string
from rake_nltk import Metric, Rake
from keybert import KeyBERT

def get_rake_keys(text):
  rake = Rake(
      stopwords=my_stopwords,
      punctuations=string.punctuation
  )
  rake.extract_keywords_from_text(text)
  return rake.get_ranked_phrases()

def get_bert_keys(text):
  model = KeyBERT('xlm-r-distilroberta-base-paraphrase-v1')
  keywords = model.extract_keywords(text, keyphrase_ngram_range=(1, 2),
                                    stop_words=nltk.corpus.stopwords.words('english'),
                                    nr_candidates=20,
                                    top_n=5,
                                    use_maxsum=True,
                                    diversity=0.3)
  return keywords

In [ ]:
grouped_reviews = json_data.groupby('asin').agg({'reviewText' : ' '.join})
test_df = grouped_reviews.iloc[0:6].copy()

RAKE model

In [ ]:
test_df['rake_keywords'] = test_df.apply(lambda row: get_rake_keys(row['reviewText']), axis = 1)

test_df

,reviewText,rake_keywords
asin,,
120401325X,They look good and stick good! I just don't li...,"[siri kept popping, multiple apple products, s..."
3998899561,it worked for the first week then it only char...,"[also needed new replacement batteries, called..."
6073894996,"Surprisingly, this inexpensive version works j...","[htc evo v 3d )., usb car charger 2 stars, hig..."
7532385086,I love this case! It's so pretty.And I love th...,"[description page kept changing, cell phone st..."
7887421268,I loved the case when I first received it but ...,"[times since putting, black leopard spots, ive..."
8199406933,"very good charger, it woks fine, no complaints...","[folding blades make, high end phones, specifi..."


BERT model

In [56]:
test_df['bert_keywords'] = test_df.apply(lambda row: get_bert_keys(row['reviewText']), axis = 1)

test_df

,reviewText,rake_keywords,bert_keywords,combined_tags,grouped_tags
asin,,,,,
120401325X,They look good and stick good! I just don't li...,"[siri kept popping, multiple apple products, s...","[look stylish, sister awesome, phone look, kep...","[look stylish, sister awesome, phone look, kep...","[<__main__.TagGroup object at 0x7f26114444e0>,..."
3998899561,it worked for the first week then it only char...,"[also needed new replacement batteries, called...","[full day, 7am return, battery charges, works ...","[full day, 7am return, battery charges, works ...","[<__main__.TagGroup object at 0x7f26113ca828>,..."
6073894996,"Surprisingly, this inexpensive version works j...","[htc evo v 3d )., usb car charger 2 stars, hig...","[griffin charging, excellent value, holds batt...","[griffin charging, excellent value, holds batt...","[<__main__.TagGroup object at 0x7f26113ca3c8>,..."
7532385086,I love this case! It's so pretty.And I love th...,"[description page kept changing, cell phone st...","[still pretty, case feels, beautiful pink, hap...","[still pretty, case feels, beautiful pink, hap...","[<__main__.TagGroup object at 0x7f26113ca940>,..."
7887421268,I loved the case when I first received it but ...,"[times since putting, black leopard spots, ive...","[cases like, case keeps, case easy, case bough...","[cases like, case keeps, case easy, case bough...","[<__main__.TagGroup object at 0x7f26113220f0>,..."
8199406933,"very good charger, it woks fine, no complaints...","[folding blades make, high end phones, specifi...","[fine complaints, charger brand, product arriv...","[fine complaints, charger brand, product arriv...","[<__main__.TagGroup object at 0x7f261149c7b8>,..."


In [ ]:
import numpy as np
import copy

def merge_tags(df_row):
  r_tags = df_row['rake_keywords'][0:15]
  b_tags = df_row['bert_keywords']
  res = copy.deepcopy(b_tags)
  for tag in r_tags:
    res.append(tag)
  return res

test_df['combined_tags'] = test_df.apply(lambda row: merge_tags(row), axis=1)
test_df

,reviewText,rake_keywords,bert_keywords,combined_tags
asin,,,,
120401325X,They look good and stick good! I just don't li...,"[siri kept popping, multiple apple products, s...","[look stylish, sister awesome, phone look, kep...","[look stylish, sister awesome, phone look, kep..."
3998899561,it worked for the first week then it only char...,"[also needed new replacement batteries, called...","[full day, 7am return, battery charges, works ...","[full day, 7am return, battery charges, works ..."
6073894996,"Surprisingly, this inexpensive version works j...","[htc evo v 3d )., usb car charger 2 stars, hig...","[griffin charging, excellent value, holds batt...","[griffin charging, excellent value, holds batt..."
7532385086,I love this case! It's so pretty.And I love th...,"[description page kept changing, cell phone st...","[still pretty, case feels, beautiful pink, hap...","[still pretty, case feels, beautiful pink, hap..."
7887421268,I loved the case when I first received it but ...,"[times since putting, black leopard spots, ive...","[cases like, case keeps, case easy, case bough...","[cases like, case keeps, case easy, case bough..."
8199406933,"very good charger, it woks fine, no complaints...","[folding blades make, high end phones, specifi...","[fine complaints, charger brand, product arriv...","[fine complaints, charger brand, product arriv..."


In [ ]:
import re
from nltk.corpus import wordnet
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
def maxSimilarity(word1, word2):
  maxSimilarity = 0
  for synsetWord1 in wordnet.synsets(word1):
        for synsetWord2 in wordnet.synsets(word2):
          similarity = synsetWord1.wup_similarity(synsetWord2)
          if(similarity != None and similarity > maxSimilarity):
            maxSimilarity = similarity
  return maxSimilarity

def numSimilarWords(tag1, tag2):
  tag1Words = re.split('\W+', tag1)
  tag2Words = re.split('\W+', tag2)
  similarityThreshold = 0.8
  numSimilarWords = 0

  for tag1Word in tag1Words:
    maxSimilarityWithAWord = 0
    for tag2Word in tag2Words:
      similarity = maxSimilarity(tag1Word, tag2Word)
      if similarity > maxSimilarityWithAWord:
        maxSimilarityWithAWord = similarity
    if maxSimilarityWithAWord > similarityThreshold:
      numSimilarWords += 1
    elif maxSimilarityWithAWord > similarityThreshold * 0.75:
      numSimilarWords += 0.25
  return numSimilarWords

def tagsAreSimilar(tag1, tag2):
  tag1Words = re.split('\W+', tag1)
  tag2Words = re.split('\W+', tag2)
  smallerTagLength = len(tag1Words) if len(tag1Words) < len(tag2Words) else len(tag2Words)
  return True if numSimilarWords(tag1, tag2) > smallerTagLength / 2 else False

In [ ]:
class TagGroup:
  def __init__(self, tags):
    self.tags = tags
    self.name = tags[0]

  def addTag(self, tag):
    self.tags.append(tag)
    

def belongsInTagGroup(tag, tagGroup):
  for t in tagGroup.tags:
    if tagsAreSimilar(t, tag):
      return True
  return False

def groupTags(tags):
  tagGroups = []
  tagsUsed = [False for x in range(len(tags))]
  for i in range(0, len(tags)):
    if tagsUsed[i] is False:
      newTagGroup = TagGroup([tags[i]])
      for j in range(i + 1, len(tags)):
        if tagsUsed[j] is False and belongsInTagGroup(tags[j], newTagGroup):
          newTagGroup.addTag(tags[j])
          tagsUsed[j] = True
      tagGroups.append(newTagGroup)
  return tagGroups

test_df
test_df['grouped_tags'] = test_df.apply(lambda row: groupTags(row['combined_tags']), axis=1)
    

In [ ]:
for group in test_df['grouped_tags'].iloc[1]:
  print(group.name)
  print(group.tags)
  print("\n")

full day
['full day', '2 whole days', 'full battery lasts', 'first week']


7am return
['7am return']


battery charges
['battery charges', 'works great', 'also needed new replacement batteries', 'called power bear ..', 'rechargeable battery cases', 'battery case came', 'powerbear case became', 'first battery case', 'power case', 'double power']


protects phone
['protects phone']


single thing happen
['single thing happen', '5 star experience']


add much weight
['add much weight']


house around 7am
['house around 7am']


